In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import dash
from dash import dcc, html, Dash, State
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.express as px

mapbox_token = 'pk.eyJ1IjoiYW5kcmVhc29zdGVkIiwiYSI6ImNsbmJxMjFndDA4dm8ybXJrMzhia2NqdnoifQ.fXDNIJ1LelhA1ypNiaJE9w'

df = pd.read_csv('Data/homicide_data.csv')
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('state')

In [2]:
df = pd.read_csv('Data/homicide_data.csv')
df['victim_full_name'] = df['victim_first'] +  ' ' + df['victim_last']

df = df.sort_values('city')

In [ ]:
help(px.scatter_mapbox)

In [4]:
def make_map(city):
    filtered_df = df[df['city'] == city] if city != 'all cities' else df

  
    fig = px.scatter_mapbox(
        filtered_df,
        lat="lat",
        lon="lon",
        hover_name="victim_full_name",
        hover_data=["victim_race", "victim_age", "victim_sex"],
        color = "disposition",
        height=700
    )
    
    title = f'Murders in {city}' if city is not None else ''

    
    fig.update_layout(mapbox_style="light", 
                      mapbox_accesstoken=mapbox_token, 
                      title = title,
                      #mapbox_center={"lat": center_lat, "lon": center_lon},  
                      #mapbox_zoom = 10,
                      autosize = True
                      )
    
    fig.update_layout(mapbox_zoom = 9 if city != 'all cities' else 3.9)

    fig.update_traces(uirevision='persist')
    
    fig.update_layout(margin={"r": 0, "l": 0, "b": 0})
    fig.update_layout(clickmode='event+select',
                      hovermode='closest')


    
    return fig

app = dash.Dash(__name__)


cityoptions=[{'label': 'Select all', 'value': 'all cities'}]+[{'label': city, 'value': city} for city in sorted(df['city'].unique())]
# Define layout
app.layout = html.Div([
    dcc.Dropdown(
        id='city_dropdown',
        options=cityoptions,
        value = 'all cities', #default
        #value = None,
        #placeholder = "Select city",
        clearable=False,
        style={'width': '200px'}
    ),
    dcc.Graph(id='murder-map'),
    html.Button('Reset Selection', id='reset-button', n_clicks=0)
])



# Define callback to update the map when dropdown selection changes
@app.callback(
    [Output('murder-map', 'figure'),
    Output('reset-button', 'n_clicks')],
    [Input('city_dropdown', 'value'),
     Input('murder-map', 'selectedData'),
     Input('reset-button', 'n_clicks')]
)
def update_map(selected_city, selected_data, n_clicks):
    fig = make_map(selected_city)
    

    if selected_data:
        latitudes = [point['lat'] for point in selected_data['points']]
        longitudes = [point['lon'] for point in selected_data['points']]
        #print(selected_data['points'])
        center = {
            "lat": np.mean(latitudes),
            "lon": np.mean(longitudes)
        }
        zoom = 12 
        
        fig.update_layout(mapbox_center=center, mapbox_zoom=zoom)

        
        selected_points = [(point['pointIndex'], point['curveNumber']) for point in selected_data['points']]
        unique_curves = range(len(fig.data)) #all possible curves (legend items )

        selected_dict = {c: [] for c in unique_curves}
        for p,c in selected_points:
            selected_dict[c].append(p)
        
        for u in unique_curves:
            fig.data[u].update(selectedpoints = selected_dict[u])

    if n_clicks is not None and n_clicks > 0:
        fig = make_map(selected_city)
        n_clicks = 0
        return fig,0

    return fig,0


# Define callback to update the map when dropdown selection changes
@app.callback(
    Output('murder-map', 'figure', allow_duplicate=True),
    [Input('city_dropdown', 'value')], 
    prevent_initial_call=True
)
def update_map_on_city_select(selected_state):
    fig = make_map(selected_state)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

